# Covid19 Investigation

Set some key parameters:

In [43]:
mortality_rate = 8  # In %; see near the end for how I determined this
days_to_first_symptom = 5 # 5.4 days is the average value I have seen cited
days_from_first_symptom_to_death = 14  # again, the average

Import dependencies:

In [44]:
from datetime import date, datetime, timedelta
import os
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import matplotlib.pyplot as plt 
import matplotlib.dates as dates
import ipywidgets as widgets
from scipy.optimize import curve_fit


register_matplotlib_converters()
%matplotlib inline

Fetch the latest updated data from the Johns Hopkins repo.

In [45]:
df = None
last = datetime.now() - timedelta(hours=1)


def get_data():
    """
    Get the latest data. Just return the cached copy
    if less than one hour has elapsed.
    """
    global last, df
    now = datetime.now()
    if df and now - last < timedelta(hours=1):
        print('Using cached copy')
        return df

    if not os.path.exists('csse_covid_19_daily_reports'):
        os.mkdir('csse_covid_19_daily_reports')
        
    last = now    
    start = date(2020, 1, 22)
    end = date.today()

    df = pd.DataFrame()
    while start <= end:
        try:
            which = f'csse_covid_19_daily_reports/{start.month:02d}-{start.day:02d}-{start.year}.csv'
            if os.path.exists(which):
                df_day = pd.read_csv(which)
            else:
                df_day = pd.read_csv(f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/{which}')
                df_day.to_csv(which)
            df_day['Date'] = start
            if 'Country_Region' in df_day.columns:
                df_day = df_day.rename(columns={'Country_Region': 'Country/Region', 'Province_State': 'Province/State'})
            df = df.append(df_day)
        except Exception as e:
            print(f'{start}: {e}')
        start += timedelta(days=1)
    
    del df['Last Update']
    df = df.replace({'Country/Region': {'Mainland China': 'China'}})
    df.fillna({'Province/State': ''}, inplace=True)
    for col in ['Confirmed', 'Deaths', 'Recovered']:
        df.fillna({col: 0}, inplace=True)
    return df

In [46]:
df = get_data()
df.tail(5)

,Active,Admin2,Combined_Key,Confirmed,Country/Region,Date,Deaths,FIPS,Last_Update,Lat,Latitude,Long_,Longitude,Province/State,Recovered,Unnamed: 0
3425,78.0,NaN,Venezuela,119.0,Venezuela,2020-03-28,2.0,NaN,2020-03-28 23:05:25,6.423800,NaN,-66.589700,NaN,,39.0,3425
3426,153.0,NaN,Vietnam,174.0,Vietnam,2020-03-28,0.0,NaN,2020-03-28 23:05:25,14.058324,NaN,108.277199,NaN,,21.0,3426
3427,79.0,NaN,West Bank and Gaza,98.0,West Bank and Gaza,2020-03-28,1.0,NaN,2020-03-28 23:05:25,31.952200,NaN,35.233200,NaN,,18.0,3427
3428,28.0,NaN,Zambia,28.0,Zambia,2020-03-28,0.0,NaN,2020-03-28 23:05:25,-13.133897,NaN,27.849332,NaN,,0.0,3428
3429,6.0,NaN,Zimbabwe,7.0,Zimbabwe,2020-03-28,1.0,NaN,2020-03-28 23:05:25,-19.015438,NaN,29.154857,NaN,,0.0,3429


Because the JH data is split by sub-regions (Province/State) we need to be able to aggregate it for countries. We also want to be able to handle it as either "daily new" or "cumulative". The helper function below can do this for us.

In [47]:
def aggregate_for_locations(df, locations, sub_locs=None, do_diffs=True, fields=None):
    if fields is None:
        fields = ['Confirmed', 'Deaths', 'Recovered']
    cols = ['Date']
    cols.extend(fields)
    
    if isinstance(locations, str):
        locations = [locations]
        
    if sub_locs:
        in_loc = df[(df['Country/Region'].isin(locations)) & (df['Province/State'].isin(sub_locs))]
    else:
        in_loc = df[df['Country/Region'].isin(locations)]
        
    result = in_loc[cols].groupby('Date').sum()
    
    if do_diffs:
        for f in fields:
            result[f] = result[f].diff()
            result.fillna({f: 0}, inplace=True)
    return result


Helper function to plot a time series chart:

In [48]:
def plot_time_series(df, title):
    fig, ax = plt.subplots()
    ax.plot_date(df.index, df, 'v-')
    ax.xaxis.set_minor_locator(dates.WeekdayLocator(byweekday=(1), interval=1))
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%d\n%a'))
    ax.xaxis.grid(True, which="minor")
    ax.yaxis.grid()
    ax.xaxis.set_major_locator(dates.MonthLocator())
    ax.xaxis.set_major_formatter(dates.DateFormatter('\n\n\n%b\n%Y'))
    ax.legend(df.columns, loc='upper left', shadow=True)
    ax.set_title(title)
    plt.tight_layout()
    return plt

Get the set of possible Country/Region values:

In [49]:
locations = sorted(list(df['Country/Region'].unique()))

We have one chart that lets us drill down into US states so we need the list of those. In some cases the data is broken into cities/counties, so we build up a dictionary of state abbreviations to the set of locations used for that state:

In [50]:
states = {
    'AL': ['Alabama'],
    'AK': ['Alaska'],
    'AZ': ['Arizona'],
    'AR': ['Arkansas'],
    'CA': ['California'],
    'CO': ['Colorado'],
    'CT': ['Connecticut'],
    'D.C.': ['District of Columbia'],
    'DE': ['Delaware'],
    'FL': ['Florida'],
    'GA': ['Georgia'],
    'HI': ['Hawaii'],
    'ID': ['Idaho'],
    'IL': ['Illinois'],
    'IN': ['Indiana'],
    'IA': ['Iowa'],
    'KS': ['Kansas'],
    'KY': ['Kentucky'],
    'LA': ['Louisiana'],
    'ME': ['Maine'],
    'MD': ['Maryland'],
    'MA': ['Massachusetts'],
    'MI': ['Michigan'],
    'MN': ['Minnesota'],
    'MS': ['Mississippi'],
    'MO': ['Missouri'],
    'MT': ['Montana'],
    'NE': ['Nebraska'],
    'NV': ['Nevada'],
    'NH': ['New Hampshire'],
    'NJ': ['New Jersey'],
    'NM': ['New Mexico'],
    'NY': ['New York'],
    'NC': ['North Carolina'],
    'ND': ['North Dakota'],
    'OH': ['Ohio'],
    'OK': ['Oklahoma'],
    'OR': ['Oregon'],
    'PA': ['Pennsylvania'],
    'PR': ['Puerto Rico'],
    'RI': ['Rhode Island'],
    'SC': ['South Carolina'],
    'SD': ['South Dakota'],
    'TN': ['Tennessee'],
    'TX': ['Texas'],
    'UT': ['Utah'],
    'VT': ['Vermont'],
    'VA': ['Virginia'],
    'WA': ['Washington'],
    'WV': ['West Virginia'],
    'WI': ['Wisconsin'],
    'WY': ['Wyoming'],
    'U.S.': [] # other territories
}

# Extend the above with all the counties/cities that were broken out separately
for s in df[df['Country/Region'] == 'US']['Province/State'].unique():
    i = s.find(',')
    if i < 0:
        continue
    st = s[i+2:i+5].strip()
    if st not in states:
        st += '.'
        if st not in states:
            print(f'Failed with {s}/{st}')
            continue
    states[st].append(s)


Create a country multi-picker:

In [51]:
picker = widgets.SelectMultiple(
                        options=locations,
                        value=['US'],
                        description='Locations'
                )

Our first interactive chart is for one or more countries, cumulative cases:

In [52]:
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, Location, do_diffs=False), 
                                                   title=f'{" ".join(Location)}: Cumulative'),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(211,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

As above, but new cases rather than cumulative:

In [53]:
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, Location), 
                                                   title=f'{" ".join(Location)}: New'),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(211,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

There are so many infections its hard to see the deaths, so pull those out separately:

In [54]:
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, Location, fields=['Deaths']), 
                                                   title=f'{" ".join(Location)}: New'),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(211,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

A chart for cumulative cases in different US states:

In [55]:
picker2 = widgets.Select(
                        options=states.keys(),
                        value='WA',
                        description='States'
                )
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, 'US', sub_locs=states[Location], do_diffs=False), 
                                                   title=f'{Location}: Cumulative'),
                 Location=picker2)

interactive(children=(Select(description='States', index=48, options=('AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT'…

<function __main__.<lambda>(Location)>

Now we use the reported deaths to try to infer the actual number infected. We first different curves to the death counts and use that to extrapolate:

In [56]:
def exponential(x, a):
    return a ** x


def power(x, a):
    return x ** a


def sigmoid(x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return (y)


def sigmoid_init(x, y):
    return [max(y), np.median(x), 1, min(y)] # initial guess


# It takes on average 5 days to show symptoms. Typically 14 days from that to death.
# So it seems reasonable to say that people infected on day x will die around day x + 19
# So if we extrapolate the death rate by 15 days, we can approximate how many are infected 
# now after adjusting for fatality rate.

def predict_from_death_rate(region=None, 
                            fatality_rate=mortality_rate, 
                            death_time=days_to_first_symptom+days_from_first_symptom_to_death, 
                            fn=exponential, 
                            init=None):
    if region is None:
        region = 'US'
        
    deaths = aggregate_for_locations(df, region, do_diffs=False)['Deaths']
    deaths = list(deaths[deaths > 0])
    
    x = range(1, len(deaths)+1)
    y = deaths
    
    if not y:
        print(f"Insufficient data for {region}")
        return
    
    if init:
        p0 = init(x, y)
        a, pcov = curve_fit(fn, x, y, p0, method='trf')
    else:
        a, pcov = curve_fit(fn, x, y)

    #a, pcov = curve_fit(fn, x[:-1], y[:-1])  # useful for seeing if latest data point is still aligned with curve

    plt.figure()
    plt.plot(x, y, 'ko', label=f"{region} Actual Deaths")
    plt.plot(x, fn(x, *a), 'r-', label="Fitted Curve")
    plt.legend()
    plt.show()
    
    forecast_deaths = fn(x[-1] + death_time, *a)
    predicted_infected = forecast_deaths * 100 / fatality_rate
    print(f'Predicted infected {int(predicted_infected)} leading to {int(forecast_deaths)} deaths in {death_time} days\nParams: {", ".join([str(aa) for aa in a])}')
    print(f'One year deaths: {fn(356, *a)}')

In [57]:
widgets.interact(lambda Location: predict_from_death_rate(Location),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(211,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

In [58]:
widgets.interact(lambda Location: predict_from_death_rate(Location, fn=power),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(211,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

In [59]:
#### Sigmoid version should be better for places that are flattening the curve or reaching saturation

widgets.interact(lambda Location: predict_from_death_rate(Location, fn=sigmoid, init=sigmoid_init),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(211,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

In [60]:
# Sanity check to make sure we loaded todays data
df[df['Date'] == date.today]

,Active,Admin2,Combined_Key,Confirmed,Country/Region,Date,Deaths,FIPS,Last_Update,Lat,Latitude,Long_,Longitude,Province/State,Recovered,Unnamed: 0


In [79]:
# Calculate actual mortality rate


def calc_mortality_rate(loc):
    data = aggregate_for_locations(df, loc)
    result = []
    end = datetime.today() + timedelta(days=1)
    end = date(end.year, end.month, end.day)
    try:
        data.loc[end]
    except Exception:
        end -= timedelta(days=1)  # don't have today yet

    for days in range(1, days_from_first_symptom_to_death + 1):
        start = end - timedelta(days=days)
        try:
            # Use a 3 day count for better average
            deaths = data.loc[end]['Deaths'] + \
                data.loc[end - timedelta(days=1)]['Deaths'] + \
                data.loc[end - timedelta(days=2)]['Deaths']

            initial = data.loc[start]['Confirmed'] + \
                data.loc[start - timedelta(days=1)]['Confirmed'] + \
                data.loc[start - timedelta(days=2)]['Confirmed']
            rate = deaths / initial if initial else 0
            result.append(
                {
                    'days': days,
                    'start': start,
                    'end': end,
                    'confirmed': initial,
                    'deaths': deaths,
                    'rate': rate
                }
            )
        except Exception:
            pass

    return pd.DataFrame(result)

widgets.interact(lambda Location: calc_mortality_rate(Location), Location=picker)


interactive(children=(SelectMultiple(description='Locations', index=(211,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>